In [188]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor

%matplotlib inline

In [189]:
names = ['API','Surf_X','Surf_Y','Date_Drilling','Date_Completion','Date_Production','Lateral_Length','Depth_TVD_PPLS','Erosion_PPLS','Pressure_PPLS','TOC_PPLS','Vcarb_PPLS','Vsand_PPLS','Vclay_PPLS','PR_PPLS','YM_PPLS','RHOB_PPLS','Res_PPLS','GR_PPLS','DT_PPLS','DTs_PPLS','Temperature','Temp_Anomaly','S3Tect_PPLS','S3_contrast_PPLS','Heat_Flow','Zone','Nbr_Stages','Frac_Gradient','Proppant_Designed','Proppant_in_Formation','Avg_Breakdown_Pressure','Avg_Treating_Pressure','Max_Treating_pressure','Min_Treating_Pressure','Avg_Rate_Slurry','Max_Rate_Slurry','Min_Rate_Slurry','ShutInPressure_Fil','ShutInPressure_Initial','ISIP','Shot_Density','Shot_Total','Proppant_per_ft','Stage_Spacing','GasCum360','OilCum360']

df_data = pd.read_csv('./TrainSample.csv', 
                        header = None, 
                        sep = ';',
                        decimal = ',',
                        names = names,
                        skiprows = 1,
                        index_col=0,
                        parse_dates = ['Date_Drilling','Date_Completion','Date_Production'],
                        dtype = {col: np.float32 for col in names}
                       )

df_test = pd.read_csv('./TestSample.csv',
                      header = None, 
                      sep = ';', 
                      decimal = ',', 
                      names = names,
                      skiprows = 1,
                      index_col=0,
                      parse_dates = ['Date_Drilling','Date_Completion','Date_Production'],
                      dtype = {col: np.float32 for col in names}
                     )



In [190]:
## Outliers

df_data.loc[746, 'Frac_Gradient'] = np.nan
df_data.loc[553, 'ISIP'] = np.nan
df_data.loc[681, 'Min_Rate_Slurry'] = np.nan
df_data.loc[424, 'Res_PPLS'] = np.nan
df_data.loc[456, 'ShutInPressure_Fil'] = np.nan
df_data.loc[723, 'Min_Treating_Pressure'] = np.nan
df_data.loc[205, 'Min_Treating_Pressure'] = np.nan

In [191]:
data_mean_gas = df_data[['GasCum360']].mean()
data_mean_oil = df_data[['OilCum360']].mean()

In [192]:
##Définition des labels

y_gas = {'GasCum360' : df_data['GasCum360']}
y_oil = {'OilCum360' : df_data['OilCum360']}

df_target_gas = pd.DataFrame(y_gas)
df_target_oil = pd.DataFrame(y_oil)

In [193]:
cols_date = ['Date_Drilling','Date_Completion','Date_Production', 'GasCum360', 'OilCum360']

df_data.drop(cols_date, 1, inplace=True)
df_test.drop(cols_date, 1, inplace=True)

In [194]:
df_data.dropna()
df_test.dropna()
df_data = df_data.fillna(df_data.mean())
df_test = df_test.fillna(df_data.mean())

In [195]:
from sklearn import cross_validation

X_train, X_test, y_train_gas, y_test_gas = cross_validation.train_test_split(df_data, df_target_gas, test_size=0.2, random_state=0)
X_train, X_test, y_train_oil, y_test_oil = cross_validation.train_test_split(df_data, df_target_oil, test_size=0.2, random_state=0)

In [196]:
## Fit regression model training

regr_gas = ElasticNet(alpha=0.02, l1_ratio=0.5, max_iter=10)
regr_oil = ElasticNet(alpha=0.018, l1_ratio=0.7, max_iter=12)

clf_gas = MultiOutputRegressor(regr_gas)
clf_oil = MultiOutputRegressor(regr_oil)

clf_gas.fit(X_train, y_train_gas)
clf_oil.fit(X_train, y_train_oil)

y_predict_gas = clf_gas.predict(X_test)
y_predict_oil = clf_oil.predict(X_test)

In [197]:
## Metrics training

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

retour = 'Erreur Moyenne absolue : {}\n\
Erreur Moyenne carré : {}\n\
R2 : {}'.format(mean_absolute_error(y_test_oil, y_predict_oil),
                mean_squared_error(y_test_oil, y_predict_oil),
                r2_score(y_test_oil, y_predict_oil)               
               )

print(retour)

Erreur Moyenne absolue : 0.414101213217
Erreur Moyenne carré : 0.324666798115
R2 : 0.619760102802


In [198]:
## Metrics training

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

retour = 'Erreur Moyenne absolue : {}\n\
Erreur Moyenne carré : {}\n\
R2 : {}'.format(mean_absolute_error(y_test_gas, y_predict_gas),
                mean_squared_error(y_test_gas, y_predict_gas),
                r2_score(y_test_gas, y_predict_gas),
               )

print(retour)

Erreur Moyenne absolue : 0.430137306452
Erreur Moyenne carré : 0.303963541985
R2 : 0.611456443476


In [199]:
## Fit regression model test data

clf_gas.fit(df_data, df_target_gas)
clf_oil.fit(df_data, df_target_oil)

y_predictFin_gas = clf_gas.predict(df_test)
y_predictFin_oil = clf_oil.predict(df_test)

In [200]:
gascumpred = []
oilcumpred = []

gascumpred = [element[0] for element in y_predictFin_gas]
oilcumpred = [element[0] for element in y_predictFin_oil]

gascumpred_mean = mean_absolute_error(y_test_gas, y_predict_gas)
oilcumpred_mean = mean_absolute_error(y_test_oil, y_predict_oil)

GasCum360_inf = gascumpred - abs(gascumpred_mean)
GasCum360_sup = gascumpred + abs(gascumpred_mean)

OilCum360_inf = oilcumpred - abs(oilcumpred_mean)
OilCum360_sup = oilcumpred + abs(oilcumpred_mean)

GasCum360_inf.tolist
GasCum360_sup.tolist

OilCum360_inf.tolist
OilCum360_sup.tolist

<function tolist>

In [201]:
y_predict_gas[:10]

array([[-0.63327819],
       [ 0.96618533],
       [-0.29333723],
       [-1.00627685],
       [-0.86099166],
       [ 0.43936178],
       [-0.215238  ],
       [-1.30247939],
       [-0.18422464],
       [-0.71621197]], dtype=float32)

In [202]:
y_test_gas.head(10)

,GasCum360
API,
521.0,-1.056089
38.0,-0.036146
471.0,-0.150183
81.0,-0.798361
237.0,-0.990523
442.0,-0.842177
375.0,0.419826
610.0,-0.879051
706.0,-0.338057


In [203]:
## Output

id_test = df_test.index.tolist()

output = pd.DataFrame({'API': id_test,
                       'GasCum360_INF': GasCum360_inf,
                       'GasCum360_SUP': GasCum360_sup,
                       'OilCum360_INF': OilCum360_inf,
                       'OilCum360_SUP': OilCum360_sup},
                      index=id_test
                     )

output.head()

output.to_csv('coche-julien-challenge-total.csv', index=False, sep= ';', decimal=',')

In [204]:
print(output.head(5))

         API  GasCum360_INF  GasCum360_SUP  OilCum360_INF  OilCum360_SUP
633.0  633.0      -0.922555      -0.062280      -0.130504       0.697698
587.0  587.0       0.019577       0.879852      -1.486853      -0.658650
264.0  264.0       0.301614       1.161888      -0.810277       0.017925
680.0  680.0       0.041204       0.901479      -1.341707      -0.513505
443.0  443.0       0.124494       0.984768      -0.424520       0.403682
